## 1. Package Preparation

In [1]:
#import packages

# clear warnings
import warnings
warnings.simplefilter("ignore")

# import keras data
from keras import backend as K
K.set_image_dim_ordering('tf') # note that we need to have tensorflow dimension ordering still because of the weigths.
print('The backend is:',K.backend())
import tensorflow as tf
print(K.image_dim_ordering()) # should say tf
print(tf.__version__) # tested for 1.11.0
import cv2
import keras
print(keras.__version__) # tested for 2.2.4

Using TensorFlow backend.


The backend is: tensorflow
tf
1.11.0
2.2.4


In [2]:
# Import relevant packages
from __future__ import absolute_import, division, print_function # make it compatible w Python 2
import os
import h5py # to handle weights
import os, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

# to read image
from PIL import Image

# relative keras packages
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation, ZeroPadding2D
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import model_from_json
from keras.preprocessing import image

# useful packages from sklearn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

from IPython.display import Image, display

# fix random seed for reproducibility
np.random.seed(150)

## 2. Read data from preprocessed pickle file

In [3]:
# We have preprocessed our data and saved in a pickle file
# Now we only need to read this pickle file
map_data = pd.read_pickle('position_df.pkl')

In [4]:
map_data.head()

,location,index,label,image
0,"(40.7215815,-73.9977854)",9,3,"[[[65, 39, 35], [65, 39, 35], [73, 47, 43], [7..."
1,"(40.7214509,-73.9973979)",11,4,"[[[65, 40, 36], [69, 36, 36], [69, 36, 36], [7..."
2,"(40.7208958,-73.9957503)",12,4,"[[[65, 51, 53], [66, 52, 79], [66, 52, 79], [7..."
3,"(40.7211736,-73.9965747)",13,4,"[[[104, 92, 100], [104, 88, 96], [100, 88, 96]..."
4,"(40.7206316,-73.9949663)",14,4,"[[[144, 136, 140], [138, 132, 137], [130, 122,..."


The column 'index' in the map_data above is used to help match the location data for the 'location' column.

We only need the data in the 'label' column and 'image' column to train our model, so we drop those two unnecessary columns.

In [5]:
map_data.drop(['location','index'],axis=1,inplace=True)

In [6]:
# Show the head of our map_data
map_data.head()

,label,image
0,3,"[[[65, 39, 35], [65, 39, 35], [73, 47, 43], [7..."
1,4,"[[[65, 40, 36], [69, 36, 36], [69, 36, 36], [7..."
2,4,"[[[65, 51, 53], [66, 52, 79], [66, 52, 79], [7..."
3,4,"[[[104, 92, 100], [104, 88, 96], [100, 88, 96]..."
4,4,"[[[144, 136, 140], [138, 132, 137], [130, 122,..."


## 3. Split data for cross validation

In [64]:
Xs = pd.DataFrame(map_data.iloc[:,1])
ys = pd.DataFrame(map_data.iloc[:,0])
ys['label'] = ys['label'].apply(lambda x: int(x))

In [65]:
# Split data into training data (including training and validation) and test data
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size = 0.2, random_state = 0)

In [66]:
# Split training data into true training data and validation data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [67]:
# confirm the shape and type of our data is right
print('The shape of the first item of X_train is', X_train.iloc[0,0].shape)
print('The length of X_train is', len(X_train))

print('The shape of the first item of X_val is', X_val.iloc[0,0].shape)
print('The length of X_val is', len(X_val))

print('The shape of the first item of X_test is', X_test.iloc[0,0].shape)
print('The length of X_test is', len(X_test))

print('The type and value of the first item of y_train is', type(y_train.iloc[0,0]), y_train.iloc[0,0])
print('The length of y_train is', len(y_train))

print('The shape of the first item of y_val is', type(y_val.iloc[0,0]), y_val.iloc[0,0])
print('The length of y_val is', len(y_val))

print('The type and value of the first item of y_test is', type(y_test.iloc[0,0]), y_test.iloc[0,0])
print('The length of y_test is', len(y_test))

The shape of the first item of X_train is (640, 640, 3)
The length of X_train is 2235
The shape of the first item of X_val is (640, 640, 3)
The length of X_val is 559
The shape of the first item of X_test is (640, 640, 3)
The length of X_test is 699
The type and value of the first item of y_train is <class 'numpy.int64'> 0
The length of y_train is 2235
The shape of the first item of y_val is <class 'numpy.int64'> 4
The length of y_val is 559
The type and value of the first item of y_test is <class 'numpy.int64'> 1
The length of y_test is 699


In [68]:
# A small check of total data amount before training the model
check_X = (len(X_train) + len(X_val) + len(X_test)) == len(map_data)
check_y = (len(y_train) + len(y_val) + len(y_test)) == len(map_data)

print('X data sets equality is:', check_X)
print('y data sets equality is:', check_y)

X data sets equality is: True
y data sets equality is: True


## 4. One-hot encoding and normalizing our input data

In [80]:
# Create combined lists for training and test sets,
# so that we can do normalization and encoding in one function on both data sets.
# Note that 'combine' is only a pointer,
# so when we change something on data sets in combine,
# the original data sets will also change.

combine_X = [X_train, X_val, X_test]
combine_y = [y_train, y_val, y_test]

In [81]:
combine_y[0]

,label
2861,0
761,4
2987,0
2236,1
2598,2
1046,3
2368,3
1201,3
401,1
2169,2


In [82]:
combine_y = [np.array(y_train['label']), np.array(y_val['label']), np.array(y_test['label'])]

In [83]:
combine_y

[array([0, 4, 0, ..., 1, 0, 2], dtype=int64),
 array([4, 2, 4, 1, 2, 4, 0, 3, 0, 1, 0, 3, 4, 3, 1, 3, 4, 2, 0, 1, 3, 4,
        1, 0, 3, 1, 4, 3, 2, 3, 3, 1, 0, 2, 1, 0, 0, 4, 4, 1, 4, 2, 0, 1,
        1, 1, 3, 4, 2, 3, 2, 0, 2, 0, 1, 3, 0, 3, 0, 4, 1, 0, 2, 3, 0, 3,
        1, 3, 3, 1, 4, 1, 4, 4, 1, 1, 3, 4, 1, 4, 2, 2, 4, 0, 0, 1, 4, 3,
        2, 1, 2, 4, 1, 4, 2, 4, 1, 3, 0, 0, 1, 3, 1, 4, 0, 0, 2, 2, 3, 2,
        4, 3, 4, 0, 3, 1, 3, 0, 2, 0, 4, 2, 0, 1, 1, 1, 0, 2, 1, 4, 1, 3,
        2, 1, 3, 2, 2, 0, 3, 2, 4, 3, 0, 2, 3, 4, 0, 0, 3, 1, 0, 3, 0, 1,
        4, 2, 2, 4, 3, 0, 1, 2, 4, 2, 3, 1, 4, 2, 2, 3, 0, 1, 3, 4, 3, 4,
        0, 1, 3, 0, 3, 1, 4, 1, 2, 1, 4, 2, 1, 1, 2, 0, 4, 2, 2, 3, 0, 3,
        1, 4, 3, 4, 4, 3, 0, 0, 0, 1, 1, 4, 1, 1, 1, 2, 3, 4, 1, 2, 2, 2,
        3, 1, 2, 2, 4, 0, 2, 1, 3, 4, 4, 3, 4, 0, 0, 1, 0, 1, 1, 4, 1, 2,
        1, 0, 1, 1, 4, 3, 1, 2, 4, 2, 2, 2, 0, 2, 3, 3, 4, 3, 1, 1, 2, 4,
        1, 2, 0, 4, 0, 3, 0, 4, 4, 4, 2, 4, 4, 2, 0, 4, 0, 2, 0, 3

In [ ]:
# normalize inputs from 0-255 to 0.0-1.0
for df in combine_X:
    df = df / 255.0

In [ ]:
# check the value of normalized data
print('The shape and value of X data after normalization:', df.iloc[0,0].shape, '\n', df.iloc[0,0][0])

In [ ]:
#### from sklearn.preprocessing imposrt LabelEncoder, OneHotEncoder

In [84]:
#### one hot encoding our categorical data
#labelencoder_y = LabelEncoder()
#enc = OneHotEncoder()
for df in combine_y:
    df = np_utils.to_categorical(df)
    #df.iloc[:,0] = labelencoder_y.fit_transform(df.iloc[:,0])
    #df = enc.fit(np.array(df).reshape(-1,1))

In [89]:
combine_y_0 = np_utils.to_categorical(combine_y[0])
combine_y[0] = combine_y_0
combine_y[1] = np_utils.to_categorical(combine_y[1])
combine_y[2] = np_utils.to_categorical(combine_y[2])

In [90]:
# Check the type of y after one-hot encoding
type(y_train.iloc[0,0])

numpy.int64

In [102]:
y_train = combine_y[0]

In [103]:
type(y_train)

numpy.ndarray

## 5. Model Training

The function below is adapted from: 
http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/

In [100]:
# First type of CNN model
input_size=(640,640,3)
num_classes=5

def createCNNModel(num_classes):

    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 640, 640, 32)      896       
_________________________________________________________________
dropout_3 (Dropout)          (None, 640, 640, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 640, 640, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 320, 320, 32)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3276800)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1677722112
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
__________

The function below is adapted from: 
http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/

In [ ]:
'''# Another type of CNN model
input_size=(640,640,3)
num_classes=6

def createCNNModel(num_classes):

    # Create the model
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=input_size, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    epochs = 3  # >>> should be 25+
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

# create our CNN model
model, epochs = createCNNModel(num_classes)
print("CNN Model created.")'''

In [95]:
X_train.shape

(2235, 1)

In [96]:
X_image_list = X_train['image'].tolist()

In [104]:
X_train_array = np.array(X_image_list)
y_train_array = np.array(y_train)

In [105]:
y_train_array

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
#batch size，learning rate can be modified before training

batch_size=60
seed = 7
np.random.seed(seed)
model.fit(X_train_array, y_train_array, batch_size = batch_size, nb_epoch = epochs)
#model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=60)

Epoch 1/3


In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

print("done")

In [ ]:
y_pred=model.predict(X_test)
print("y_pred is ",y_pred)

In [ ]:
y_test

In [ ]:
##store the weights
for layer in model.layers:
    weights = layer.get_weights() 

In [ ]:
weights

In [ ]:
len(weights)

In [ ]:
## store the model
model.save('cnn_trial.h5')

In [ ]:
###read the stored the model
import h5py

def print_keras_wegiths(weight_file_path):
    f = h5py.File(weight_file_path)  # 读取weights h5文件返回File类
    try:
        if len(f.attrs.items()):
            print("{} contains: ".format(weight_file_path))
            print("Root attributes:")
        for key, value in f.attrs.items():
            print("  {}: {}".format(key, value))  # 输出储存在File类中的attrs信息，一般是各层的名称

        for layer, g in f.items():  # 读取各层的名称以及包含层信息的Group类
            print("  {}".format(layer))
            print("    Attributes:")
            for key, value in g.attrs.items(): # 输出储存在Group类中的attrs信息，一般是各层的weights和bias及他们的名称
                print("      {}: {}".format(key, value))  

            print("    Dataset:")
            for name, d in g.items(): # 读取各层储存具体信息的Dataset类
                print("      {}: {}".format(name, d.value.shape)) # 输出储存在Dataset中的层名称和权重，也可以打印dataset的attrs，但是keras中是空的
                print("      {}: {}".format(name. d.value))
    finally:
        f.close()